In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import tacotron2
import malaya_speech.config
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [4]:
tacotron2_config = malaya_speech.config.tacotron2_config

In [5]:
model = tacotron2.Model(tacotron2.Config(vocab_size = 20, **tacotron2_config), training = True)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
input_ids = tf.placeholder(tf.int32, [None, None])
input_lengths = tf.placeholder(tf.int32, [None])
speaker_ids = tf.constant([0], dtype = tf.int32)
mel_outputs = tf.placeholder(tf.float32, [None, None, 80])
mel_lengths = tf.placeholder(tf.int32, [None])
mel_actuals = tf.placeholder(tf.float32, [None, None, 80])

In [7]:
guided = tacotron2.generate_guided_attention(mel_lengths[0], input_lengths[0])
guided = tf.expand_dims(guided, 0)
guided

<tf.Tensor 'ExpandDims:0' shape=(1, ?, ?) dtype=float32>

In [8]:
r = model(input_ids, input_lengths, speaker_ids, mel_outputs, mel_lengths, training = True)
r

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(<tf.Tensor 'model/Reshape:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'model/add:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'model/Reshape_1:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'model/transpose:0' shape=(?, ?, ?) dtype=float32>)

In [9]:
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss

In [10]:
binary_crossentropy = tf.keras.losses.BinaryCrossentropy(
    from_logits=True
)
mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()
decoder_output, post_mel_outputs, stop_token_predictions, alignment_histories = r
mel_loss_before = calculate_3d_loss(
    mel_actuals, decoder_output, loss_fn=mae
)
mel_loss_after = calculate_3d_loss(
    mel_actuals, post_mel_outputs, loss_fn=mae
)
max_mel_length = tf.reduce_max(mel_lengths)
stop_gts = tf.expand_dims(
    tf.range(tf.reduce_max(max_mel_length), dtype=tf.int32), 0
)
stop_gts = tf.tile(
    stop_gts, [tf.shape(mel_lengths)[0], 1]
)
stop_gts = tf.cast(
    tf.math.greater_equal(stop_gts, tf.expand_dims(mel_lengths, 1)),
    tf.float32,
)
stop_token_loss = calculate_2d_loss(
    stop_gts, stop_token_predictions, loss_fn=binary_crossentropy
)
attention_masks = tf.cast(
    tf.math.not_equal(guided, -1.0), tf.float32
)
loss_att = tf.reduce_sum(
    tf.abs(alignment_histories * guided) * attention_masks,
    axis=[1, 2],
)
loss_att /= tf.reduce_sum(attention_masks, axis=[1, 2])

In [12]:
loss_att

<tf.Tensor 'truediv_3:0' shape=(?,) dtype=float32>

In [13]:
loss = stop_token_loss + mel_loss_before + mel_loss_after + loss_att

In [22]:
stop_token_loss, mel_loss_before, mel_loss_after, loss_att

(<tf.Tensor 'binary_crossentropy/weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'mean_absolute_error/weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'mean_absolute_error_1/weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'truediv_3:0' shape=(?,) dtype=float32>)

In [14]:
# r = model.inference(input_ids, input_lengths, speaker_ids)
# r

In [15]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [16]:
sess.run(r, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]})

(array([[[-0.05845664,  0.01698914, -0.03797201, ..., -0.12859719,
           0.11696506, -0.06098603],
         [-0.03344746,  0.04534059, -0.02232369, ..., -0.1299415 ,
           0.12598631, -0.08066774],
         [-0.0208192 ,  0.07812613, -0.00713367, ..., -0.12329363,
           0.13041943, -0.10039776],
         ...,
         [ 0.05588069,  0.1256418 ,  0.13779905, ..., -0.06631085,
           0.17549565, -0.17695418],
         [ 0.06054533,  0.12437256,  0.13191333, ..., -0.0585171 ,
           0.1684362 , -0.18723233],
         [ 0.05715641,  0.1180123 ,  0.12852316, ..., -0.06226904,
           0.17242554, -0.19674388]]], dtype=float32),
 array([[[-0.8391762 ,  0.30608574, -0.5190792 , ..., -0.03872657,
           1.4743817 ,  0.7048205 ],
         [ 0.9585729 ,  0.310763  , -1.6525022 , ..., -0.59665275,
           0.88983357,  0.53238136],
         [ 0.31971854,  1.2167671 , -2.3504717 , ..., -0.47918493,
           3.0840304 , -0.9284392 ],
         ...,
         [-3.00398

In [18]:
sess.run(loss, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_actuals: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]})

array([3.0031924], dtype=float32)

In [19]:
sess.run(loss, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9],
                                       [1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9, 9],
                        mel_outputs: np.random.normal(size=(2, 50, 80)).astype(np.float32),
                        mel_actuals: np.random.normal(size=(2, 50, 80)).astype(np.float32),
                        mel_lengths: [50, 50]})

array([3.0227776, 3.022749 ], dtype=float32)

In [20]:
sess.run(guided, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_actuals: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]}).shape

(1, 9, 50)